In [1]:
import numpy as np
from collections import defaultdict

In [2]:
class SVDpp:
    def __init__(self, num_users, num_items, num_factors=20, lr=0.005, reg=0.02, epochs=10, seed=42):
        self.num_users = num_users
        self.num_items = num_items
        self.num_factors = num_factors
        self.lr = lr
        self.reg = reg
        self.epochs = epochs
        self.seed = seed

        np.random.seed(self.seed)  # 🔐 Ensures consistent results

        self.mu = 0  # Global mean

        # Biases
        self.bu = np.zeros(num_users)
        self.bi = np.zeros(num_items)

        # Latent factor matrices
        self.pu = np.random.normal(0, 0.1, (num_users, num_factors))
        self.qi = np.random.normal(0, 0.1, (num_items, num_factors))
        self.yj = np.random.normal(0, 0.1, (num_items, num_factors))

        self.user_rated_items = defaultdict(set)

    def fit(self, ratings):
        self.mu = np.mean([r for (_, _, r) in ratings])

        for u, i, _ in ratings:
            self.user_rated_items[u].add(i)

        for epoch in range(self.epochs):
            for u, i, r_ui in ratings:
                rated_items = self.user_rated_items[u]
                sqrt_Nu = np.sqrt(len(rated_items)) if rated_items else 1.0

                sum_y = np.sum(self.yj[list(rated_items)], axis=0) if rated_items else np.zeros(self.num_factors)
                sum_y /= sqrt_Nu

                dot = np.dot(self.qi[i], self.pu[u] + sum_y)
                r_hat = self.mu + self.bu[u] + self.bi[i] + dot
                e_ui = r_ui - r_hat

                self.bu[u] += self.lr * (e_ui - self.reg * self.bu[u])
                self.bi[i] += self.lr * (e_ui - self.reg * self.bi[i])

                pu_old = self.pu[u].copy()
                qi_old = self.qi[i].copy()

                self.qi[i] += self.lr * (e_ui * (pu_old + sum_y) - self.reg * qi_old)
                self.pu[u] += self.lr * (e_ui * qi_old - self.reg * pu_old)

                for j in rated_items:
                    self.yj[j] += self.lr * (e_ui * qi_old / sqrt_Nu - self.reg * self.yj[j])

            print(f"Epoch {epoch + 1}/{self.epochs} completed.")

    def predict(self, u, i):
        rated_items = self.user_rated_items[u]
        sqrt_Nu = np.sqrt(len(rated_items)) if rated_items else 1.0

        sum_y = np.sum(self.yj[list(rated_items)], axis=0) if rated_items else np.zeros(self.num_factors)
        sum_y /= sqrt_Nu

        dot = np.dot(self.qi[i], self.pu[u] + sum_y)
        r_hat = self.mu + self.bu[u] + self.bi[i] + dot
        return r_hat

    def predict_batch(self, user_item_pairs):
        return [self.predict(u, i) for u, i in user_item_pairs]


In [3]:
ratings = [
    (0, 0, 5), (0, 1, 3),
    (1, 0, 4), (1, 2, 2),
    (2, 1, 4), (2, 2, 5),
]

num_users = 3
num_items = 3


In [4]:
model = SVDpp(num_users, num_items, epochs=10, seed=42)
model.fit(ratings)
predicted_rating = model.predict(0, 2)
print(f"\nPredicted rating for user 0 on item 2: {predicted_rating:.3f}")

Epoch 1/10 completed.
Epoch 2/10 completed.
Epoch 3/10 completed.
Epoch 4/10 completed.
Epoch 5/10 completed.
Epoch 6/10 completed.
Epoch 7/10 completed.
Epoch 8/10 completed.
Epoch 9/10 completed.
Epoch 10/10 completed.

Predicted rating for user 0 on item 2: 3.748


In [5]:
rating_values = [r for (_, _, r) in ratings]
# Calculate the global average
global_avg = sum(rating_values) / len(rating_values)
print(f"Global average rating: {global_avg:.3f}")

Global average rating: 3.833
